# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [21]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [22]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis =1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [23]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state =1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [34]:
# Create a StandardScaler instance
scaler = StandardScaler().fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)

# Fit the scaler to the features training dataset
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [35]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [36]:
# Define the number of hidden nodes for the first hidden layer
first_layer = tf.keras.layers.Dense(units=6, activation='relu', input_dim=input_nodes)

# Define the number of hidden nodes for the second hidden layer
second_layer = tf.keras.layers.Dense(units=3, activation='relu')

# Define the number of neurons in the output layer
output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')

In [37]:
# Create the Sequential model instance
model = tf.keras.models.Sequential()

# Add the first hidden layer
model.add(first_layer)

# Add the second hidden layer
model.add(second_layer)

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(output_layer)

In [38]:
# Display the Sequential model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 6)                 72        
                                                                 
 dense_8 (Dense)             (None, 3)                 21        
                                                                 
 dense_9 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [39]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
# Fit the model using 50 epochs and the training data
fit_model = model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
38/38 [==============================] - 2s 9ms/step - loss: 0.7191 - accuracy: 0.5054
Epoch 2/100
38/38 [==============================] - 0s 7ms/step - loss: 0.6926 - accuracy: 0.5588
Epoch 3/100
38/38 [==============================] - 0s 4ms/step - loss: 0.6691 - accuracy: 0.6172
Epoch 4/100
38/38 [==============================] - 0s 5ms/step - loss: 0.6443 - accuracy: 0.6430
Epoch 5/100
38/38 [==============================] - 0s 4ms/step - loss: 0.6177 - accuracy: 0.6656
Epoch 6/100
38/38 [==============================] - 0s 5ms/step - loss: 0.5939 - accuracy: 0.6906
Epoch 7/100
38/38 [==============================] - 0s 7ms/step - loss: 0.5733 - accuracy: 0.6981
Epoch 8/100
38/38 [==============================] - 0s 6ms/step - loss: 0.5588 - accuracy: 0.7048
Epoch 9/100
38/38 [==============================] - 0s 4ms/step - loss: 0.5476 - accuracy: 0.7148
Epoch 10/100
38/38 [==============================] - 0s 4ms/step - loss: 0.5393 - accuracy: 0.7239
Epoch 11/

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [41]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

13/13 - 1s - loss: 0.5120 - accuracy: 0.7525 - 687ms/epoch - 53ms/step
Loss: 0.5119568109512329, Accuracy: 0.7524999976158142


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [42]:
# Set the model's file path
file_path = Path("/student_loans.keras")

# Export your model to a keras file
model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [43]:
# Set the model's file path
file_path = Path("/student_loans.keras")

# Load the model to a new object
import_model = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [44]:
# Make predictions with the test data
predictions = import_model.predict(X_test_scaled)

# Display a sample of the predictions
predictions[0:5]

13/13 [==============================] - 0s 5ms/step


array([[0.57133734],
       [0.2857124 ],
       [0.8514304 ],
       [0.6998321 ],
       [0.99187815]], dtype=float32)

In [45]:
# Save the predictions to a DataFrame and round the predictions to binary results
import numpy as np
pred = np.ravel(predictions)
predictions_df = pd.DataFrame(columns=['predictions'], data=pred)
predictions_df['predictions'] = round(predictions_df['predictions'],0)

predictions_df

,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [46]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df['predictions'].values))

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       188
           1       0.79      0.73      0.76       212

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.76      0.75      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

The data you need would be relevant past payment history. What is the degree on, in terms to forecast possible future earnings. If its a study that is higher earning is less risk of a loan. GPA could be relevant to compare with other past people and see if that was a factor in terms of earning potential and job placement.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

You would use collaborative filtering, because you want to use past actions by other borrowers with the same situation to factor into the loan risk.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

Challenges is the ever changing economic landscape and rising tuition costs, as well as the flow of job openings in different industries, it could be that an industry in the past that had no issues in terms of job prospects of earnings has changed in the current landscape and that data is no longer representative of the times.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

